In [6]:
from pyBADA.bada4 import Bada4Aircraft, BADA4
from pyBADA.aircraft import Airplane
import pyBADA.atmosphere as atm
import math
import pandas as pd
import numpy as np

# 自定义转换函数
class conv:
    @staticmethod
    def ft2m(feet):
        return feet * 0.3048
    
    @staticmethod
    def m2ft(meters):
        return meters / 0.3048
    
    @staticmethod
    def kt2ms(knots):
        return knots * 0.51444
    
    @staticmethod
    def ms2kt(ms):
        return ms / 0.51444

def initialize_aircraft_model(ac_model):
    """初始化飞机模型和BADA4对象"""
    bada_version = "4.2"
    filepath = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"
    
    AC = Bada4Aircraft(
        badaVersion=bada_version,
        acName=ac_model,
        filePath=filepath,
    )
    
    bada4 = BADA4(AC)
    return bada4

def determine_speed_profile(flight_level, crossover_fl, descent_mach, high_cas, 
                           low_cas, low_cas_fl, constant_cas):
    """确定给定高度层的速度剖面参数"""
    if constant_cas is None and flight_level >= crossover_fl:
        # 使用指定马赫数
        M = descent_mach
        speed_mode = f"Mach {descent_mach}"
        flight_evolution = "constM"
        target_cas = None
    elif flight_level >= low_cas_fl:
        # 使用高高度CAS
        M = None  # 将在后续计算
        speed_mode = f"CAS {high_cas}kt"
        flight_evolution = "constCAS"
        target_cas = high_cas
    else:
        # 使用低高度CAS
        M = None  # 将在后续计算
        speed_mode = f"CAS {low_cas}kt"
        flight_evolution = "constCAS"
        target_cas = low_cas
    
    return M, speed_mode, flight_evolution, target_cas

def calculate_performance_at_altitude(flight_level, aircraft_mass, bada4, 
                                    crossover_fl, descent_mach, high_cas, 
                                    low_cas, low_cas_fl, constant_cas, DeltaTemp=0):
    """计算指定高度层的下降性能"""
    altitude_ft = flight_level * 100
    altitude_m = conv.ft2m(altitude_ft)
    
    # 计算大气参数
    theta_val, delta_val, sigma_val = atm.atmosphereProperties(altitude_m, DeltaTemp)
    
    # 确定速度剖面参数
    M, speed_mode, flight_evolution, target_cas = determine_speed_profile(
        flight_level, crossover_fl, descent_mach, high_cas, 
        low_cas, low_cas_fl, constant_cas
    )
    
    # 计算实际的马赫数、CAS、TAS
    if M is not None:
        # 马赫数已知，计算CAS和TAS
        tas = atm.mach2Tas(M, theta_val)
        cas = atm.mach2Cas(M, theta_val, delta_val, sigma_val)
    else:
        # CAS已知，计算马赫数和TAS
        cas = conv.kt2ms(target_cas)
        tas = atm.cas2Tas(cas, delta_val, sigma_val)
        M = atm.tas2Mach(tas, theta_val)
    
    # 动态计算能量分配因子
    esf_des = Airplane.esf(
        h=altitude_m,
        DeltaTemp=DeltaTemp,
        flightEvolution=flight_evolution,
        phase="des",
        M=M
    )
    
    # 计算升力系数
    cl = bada4.CL(delta=delta_val, mass=aircraft_mass, M=M)
    
    # 计算阻力系数(干净构型)
    HLid = 0.0  # 无襟翼
    LG = "LGUP"  # 起落架收起
    cd = bada4.CD(HLid=HLid, LG=LG, CL=cl, M=M)
    
    # 计算阻力
    drag = bada4.D(delta=delta_val, M=M, CD=cd)
    
    # 计算怠速推力
    idle_thrust = bada4.Thrust(
        delta=delta_val,
        theta=theta_val,
        M=M,
        rating="LIDL",  # 怠速设置
        DeltaTemp=DeltaTemp
    )
    
    # 计算燃油流量 (kg/s)
    fuel_flow = bada4.ff(
        delta=delta_val,
        theta=theta_val,
        M=M,
        rating="LIDL",  # 怠速设置
        DeltaTemp=DeltaTemp
    )
    
    # 计算怠速下降率
    idle_descent_rate = bada4.ROCD(
        T=idle_thrust,
        D=drag,
        v=tas,
        mass=aircraft_mass,
        ESF=esf_des,
        h=altitude_m,
        DeltaTemp=DeltaTemp
    )
    
    # 计算下降角(度)
    if tas > 0:
        descent_angle = math.degrees(math.asin(idle_descent_rate / tas))
    else:
        descent_angle = 0
    
    # 计算下降梯度(百分比)
    descent_gradient = 100 * abs(idle_descent_rate / tas) if tas > 0 else 0
    
    # 计算高距离
    ft_per_nm = abs(idle_descent_rate * 196.85 / conv.ms2kt(tas) * 60) if conv.ms2kt(tas) > 0 else 0
    
    return {
        "FL": flight_level,
        "高度(ft)": altitude_ft,
        "速度模式": speed_mode,
        "马赫数": round(M, 3),
        "CAS(kt)": round(conv.ms2kt(cas), 1),
        "TAS(kt)": round(conv.ms2kt(tas), 1),
        "下降率(ft/min)": round(idle_descent_rate * 196.85, 0),
        "下降率(m/s)": round(idle_descent_rate, 2),
        "下降角(度)": round(descent_angle, 2),
        "下降梯度(%)": round(descent_gradient, 2),
        "高距离(ft/nm)": round(ft_per_nm, 0),
        "ESF": round(esf_des, 3),
        "阻力(N)": round(drag, 0),
        "怠速推力(N)": round(idle_thrust, 0),
        "燃油流量(kg/h)": round(fuel_flow * 3600, 1),
        "燃油流量(kg/s)": round(fuel_flow, 4),
        "升力系数": round(cl, 3),
        "阻力系数": round(cd, 4)
    }

def generate_flight_levels(cruise_fl, target_fl):
    """生成下降过程中的高度层列表"""
    flight_levels = []
    for fl in range(cruise_fl, target_fl-1, -10):
        flight_levels.append(fl)
    if target_fl not in flight_levels:
        flight_levels.append(target_fl)
    return flight_levels

def calculate_cumulative_data(df, flight_levels):
    """计算累积距离、时间和燃油消耗"""
    horizontal_distance = np.zeros(len(flight_levels))
    fuel_consumption = np.zeros(len(flight_levels))
    time_elapsed = np.zeros(len(flight_levels))
    
    for i in range(1, len(flight_levels)):
        # 计算相邻高度层之间的高度差(英尺)
        altitude_diff = (flight_levels[i-1] - flight_levels[i]) * 100
        # 使用两个点的平均高距离比
        avg_ft_per_nm = (df.iloc[i-1]['高距离(ft/nm)'] + df.iloc[i]['高距离(ft/nm)']) / 2
        if avg_ft_per_nm > 0:
            segment_distance = altitude_diff / avg_ft_per_nm
        else:
            segment_distance = 0
        # 累积水平距离
        horizontal_distance[i] = horizontal_distance[i-1] + segment_distance
        
        # 计算时间和燃油
        avg_tas_kt = (df.iloc[i-1]['TAS(kt)'] + df.iloc[i]['TAS(kt)']) / 2
        avg_tas_nm_per_sec = avg_tas_kt / 3600  # 转换为 nm/s
        if avg_tas_nm_per_sec > 0:
            segment_time_seconds = segment_distance / avg_tas_nm_per_sec
        else:
            segment_time_seconds = 0
        
        time_elapsed[i] = time_elapsed[i-1] + segment_time_seconds
        
        avg_fuel_flow = (df.iloc[i-1]['燃油流量(kg/s)'] + df.iloc[i]['燃油流量(kg/s)']) / 2
        segment_fuel = avg_fuel_flow * segment_time_seconds
        fuel_consumption[i] = fuel_consumption[i-1] + segment_fuel
    
    return horizontal_distance, time_elapsed, fuel_consumption

def create_speed_profile_name(constant_cas, high_cas, low_cas, low_cas_fl, descent_mach):
    """创建速度剖面名称"""
    if constant_cas is not None:
        if high_cas == low_cas:
            return f"{high_cas}kt constant"
        else:
            return f"{high_cas}kt→{low_cas}kt@FL{low_cas_fl}"
    else:
        if high_cas == low_cas:
            return f"{descent_mach}M/{high_cas}kt"
        else:
            return f"{descent_mach}M/{high_cas}kt/{low_cas}kt@FL{low_cas_fl}"

def calculate_basic_descent_profile(cruise_fl, target_fl, aircraft_mass, 
                                  descent_mach, high_cas, ac_model, 
                                  low_cas=None, low_cas_fl=100, 
                                  constant_cas=None, print_details=True):
    """
    计算基础下降剖面（不包含减速段）
    """
    # 初始化飞机模型
    bada4 = initialize_aircraft_model(ac_model)
    
    # 设置低高度CAS
    if low_cas is None:
        low_cas = high_cas
    
    # 为恒定CAS下降设置参数
    if constant_cas is not None:
        descent_mach = 0.9  # 设置一个较高的马赫数，确保过渡高度远高于巡航高度
        high_cas = constant_cas
        low_cas = constant_cas if low_cas is None else low_cas
    
    # 计算过渡高度
    high_cas_ms = conv.kt2ms(high_cas)
    crossover_m = atm.crossOver(cas=high_cas_ms, Mach=descent_mach)
    crossover_ft = conv.m2ft(crossover_m)
    crossover_fl = int(crossover_ft / 100)
    
    # 创建速度剖面名称
    profile_name = create_speed_profile_name(constant_cas, high_cas, low_cas, low_cas_fl, descent_mach)
    
    if print_details:
        print(f"\n{'='*80}")
        print(f"基础下降剖面分析: {ac_model}")
        print(f"{'='*80}")
        print(f"飞机重量: {aircraft_mass/1000:.1f} 吨")
        print(f"下降剖面: {profile_name}")
        print(f"巡航高度: FL{cruise_fl} ({cruise_fl*100:,} ft)")
        print(f"目标高度: FL{target_fl} ({target_fl*100:,} ft)")
        print(f"过渡高度: FL{crossover_fl} ({crossover_fl*100:,} ft)")
        print(f"{'='*80}")
    
    # 生成高度层列表
    flight_levels = generate_flight_levels(cruise_fl, target_fl)
    
    # 计算每个高度层的性能
    results = []
    for fl in flight_levels:
        result = calculate_performance_at_altitude(
            fl, aircraft_mass, bada4, crossover_fl, descent_mach, 
            high_cas, low_cas, low_cas_fl, constant_cas
        )
        results.append(result)
    
    # 创建DataFrame
    df = pd.DataFrame(results)
    
    # 计算累积数据
    horizontal_distance, time_elapsed, fuel_consumption = calculate_cumulative_data(df, flight_levels)
    
    # 添加累积数据到DataFrame
    df['累积距离(nm)'] = [round(d, 1) for d in horizontal_distance]
    df['累积时间(s)'] = [round(t, 0) for t in time_elapsed]
    df['累积燃油(kg)'] = [round(f, 1) for f in fuel_consumption]
    
    if print_details:
        print_basic_profile_table(df, profile_name)
    
    # 计算总结信息
    total_distance = horizontal_distance[-1]
    total_time = time_elapsed[-1]
    total_fuel = fuel_consumption[-1]
    total_altitude_change = (cruise_fl - target_fl) * 100
    avg_ft_per_nm = total_altitude_change / total_distance if total_distance > 0 else 0
    
    summary = {
        "Profile": profile_name,
        "Descent Distance (nm)": round(total_distance, 1),
        "Descent Time (s)": round(total_time, 0),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time/3600), 1) if total_time > 0 else 0,
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/total_distance*100, 2) if total_distance > 0 else 0,
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }
    
    if print_details:
        print_summary(summary)
    
    return summary, df, flight_levels

def calculate_deceleration_segment(current_cas, target_cas, fuel_flow_kg_per_sec):
    """
    计算单个减速段所需的距离、时间和燃油
    
    参数:
    current_cas: 当前CAS (kt)
    target_cas: 目标CAS (kt) 
    fuel_flow_kg_per_sec: 燃油流量 (kg/s)
    
    返回:
    (减速距离(nm), 减速时间(s), 减速燃油(kg))
    """
    if current_cas <= target_cas:
        return 0, 0, 0
    
    # 根据要求：时间是每节1秒，距离是每节0.1海里
    decel_distance_nm = (current_cas - target_cas) / 10
    decel_time_sec = (current_cas - target_cas)  # 每节1秒
    decel_fuel = fuel_flow_kg_per_sec * decel_time_sec
    
    return decel_distance_nm, decel_time_sec, decel_fuel

def find_altitude_index(flight_levels, target_fl):
    """查找目标高度层附近的索引"""
    for i in range(1, len(flight_levels)):
        if flight_levels[i-1] > target_fl >= flight_levels[i]:
            return i
    return None

def apply_fl100_deceleration(decel_segments, new_df, flight_levels, 
                           horizontal_distance, time_elapsed, fuel_consumption):
    """应用FL100法定限速减速段"""
    fl100_index = find_altitude_index(flight_levels, 100)
    
    if fl100_index is not None:
        current_cas = new_df.iloc[fl100_index-1]['CAS(kt)']
        
        if current_cas > 250:
            fuel_flow = new_df.iloc[fl100_index-1]['燃油流量(kg/s)']
            
            decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                current_cas, 250, fuel_flow
            )
            
            decel_segments.append({
                'type': '法定减速',
                'FL': 100,
                '减速前CAS': current_cas,
                '减速后CAS': 250,
                '减速距离(nm)': decel_dist,
                '减速时间(s)': decel_time,
                '减速燃油(kg)': decel_fuel,
                'TAS(kt)': new_df.iloc[fl100_index-1]['TAS(kt)']
            })
            
            # 更新FL100及以下所有点的CAS值（新增）
            for i in range(fl100_index-1, len(new_df)):
                if new_df.iloc[i]['CAS(kt)'] > 250:
                    new_df.at[i, 'CAS(kt)'] = 250
            
            # 在FL100之后的所有点添加减速段的距离、时间和燃油
            horizontal_distance[fl100_index:] += decel_dist
            time_elapsed[fl100_index:] += decel_time
            fuel_consumption[fl100_index:] += decel_fuel
    
    return fl100_index

def apply_profile_deceleration(decel_segments, new_df, flight_levels, high_cas, low_cas, 
                             low_cas_fl, fl100_index, horizontal_distance, time_elapsed, fuel_consumption):
    """应用剖面减速段"""
    if low_cas is not None and low_cas_fl is not None and low_cas != high_cas:
        low_cas_index = find_altitude_index(flight_levels, low_cas_fl)
        
        # 避免重复计算FL100的减速
        if low_cas_index is not None and (low_cas_fl != 100 or fl100_index is None):
            current_cas = new_df.iloc[low_cas_index-1]['CAS(kt)']
            
            if current_cas > low_cas:
                fuel_flow = new_df.iloc[low_cas_index-1]['燃油流量(kg/s)']
                
                decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                    current_cas, low_cas, fuel_flow
                )
                
                decel_segments.append({
                    'type': '剖面减速',
                    'FL': low_cas_fl,
                    '减速前CAS': current_cas,
                    '减速后CAS': low_cas,
                    '减速距离(nm)': decel_dist,
                    '减速时间(s)': decel_time,
                    '减速燃油(kg)': decel_fuel,
                    'TAS(kt)': new_df.iloc[low_cas_index-1]['TAS(kt)']
                })
                
                horizontal_distance[low_cas_index:] += decel_dist
                time_elapsed[low_cas_index:] += decel_time
                fuel_consumption[low_cas_index:] += decel_fuel

def apply_approach_deceleration(decel_segments, new_df, flight_levels, 
                              horizontal_distance, time_elapsed, fuel_consumption):
    """应用FL30最终进近减速段"""
    fl030_index = find_altitude_index(flight_levels, 30)
    
    if fl030_index is not None:
        current_cas = new_df.iloc[fl030_index-1]['CAS(kt)']
        
        if current_cas > 220:
            fuel_flow = new_df.iloc[fl030_index-1]['燃油流量(kg/s)']
            
            decel_dist, decel_time, decel_fuel = calculate_deceleration_segment(
                current_cas, 220, fuel_flow
            )
            
            decel_segments.append({
                'type': '进近减速',
                'FL': 30,
                '减速前CAS': current_cas,
                '减速后CAS': 220,
                '减速距离(nm)': decel_dist,
                '减速时间(s)': decel_time,
                '减速燃油(kg)': decel_fuel,
                'TAS(kt)': new_df.iloc[fl030_index-1]['TAS(kt)']
            })
            
            horizontal_distance[fl030_index:] += decel_dist
            time_elapsed[fl030_index:] += decel_time
            fuel_consumption[fl030_index:] += decel_fuel

def apply_deceleration_segments(basic_summary, basic_df, flight_levels, 
                               high_cas, low_cas=None, low_cas_fl=100, 
                               print_details=True):
    """
    在基础下降剖面上应用所有减速段
    """
    # 复制原始数据
    new_df = basic_df.copy()
    
    # 获取原始的累积数据
    horizontal_distance = new_df['累积距离(nm)'].values.copy()
    time_elapsed = new_df['累积时间(s)'].values.copy()
    fuel_consumption = new_df['累积燃油(kg)'].values.copy()
    
    # 创建减速段记录列表
    decel_segments = []
    
    # 应用各种减速段
    fl100_index = apply_fl100_deceleration(decel_segments, new_df, flight_levels, 
                                         horizontal_distance, time_elapsed, fuel_consumption)
    
    apply_profile_deceleration(decel_segments, new_df, flight_levels, high_cas, low_cas, 
                             low_cas_fl, fl100_index, horizontal_distance, time_elapsed, fuel_consumption)
    
    apply_approach_deceleration(decel_segments, new_df, flight_levels, 
                              horizontal_distance, time_elapsed, fuel_consumption)
    
    # 更新DataFrame中的累积数据
    new_df['累积距离(nm)'] = [round(d, 1) for d in horizontal_distance]
    new_df['累积时间(s)'] = [round(t, 0) for t in time_elapsed]
    new_df['累积燃油(kg)'] = [round(f, 1) for f in fuel_consumption]
    
    if print_details:
        print_deceleration_segments_table(decel_segments)
        print_final_profile_table(new_df)
    
    # 计算新的总结信息
    total_distance = horizontal_distance[-1]
    total_time = time_elapsed[-1]
    total_fuel = fuel_consumption[-1]
    total_altitude_change = (flight_levels[0] - flight_levels[-1]) * 100
    avg_ft_per_nm = total_altitude_change / total_distance if total_distance > 0 else 0
    
    new_summary = {
        "Profile": basic_summary["Profile"] + " (含减速段)",
        "Descent Distance (nm)": round(total_distance, 1),
        "Descent Time (s)": round(total_time, 0),
        "Fuel Consumption (kg)": round(total_fuel, 1),
        "Average Fuel Flow (kg/h)": round(total_fuel/(total_time/3600), 1) if total_time > 0 else 0,
        "Average Descent Gradient (%)": round(total_altitude_change/6076.12/total_distance*100, 2) if total_distance > 0 else 0,
        "Altitude-to-Distance Ratio (ft/nm)": round(avg_ft_per_nm, 1)
    }
    
    if print_details:
        print_final_summary(new_summary, basic_summary)
    
    return new_summary, new_df, decel_segments

def print_basic_profile_table(df, profile_name):
    """打印基础下降剖面表格"""
    print(f"\n基础下降数据 - {profile_name}:")
    print("-" * 165)
    
    # 选择要显示的关键列
    display_columns = [
        'FL', '速度模式', '马赫数', 'CAS(kt)', 'TAS(kt)', 
        '下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)',
        '燃油流量(kg/h)', '累积距离(nm)', '累积时间(s)', '累积燃油(kg)'
    ]
    
    # 打印表头
    header = ""
    for col in display_columns:
        if col in ['FL', '马赫数', 'CAS(kt)', 'TAS(kt)']:
            header += f"{col:>8}"
        elif col in ['速度模式']:
            header += f"{col:>12}"
        elif col in ['下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)', '燃油流量(kg/h)']:
            header += f"{col:>12}"
        else:
            header += f"{col:>14}"
    print(header)
    print("-" * 165)
    
    # 打印数据行
    for _, row in df.iterrows():
        line = ""
        for col in display_columns:
            value = row[col]
            if col in ['FL']:
                line += f"{value:>8.0f}"
            elif col in ['马赫数']:
                line += f"{value:>8.3f}"
            elif col in ['CAS(kt)', 'TAS(kt)']:
                line += f"{value:>8.1f}"
            elif col in ['速度模式']:
                line += f"{str(value):>12}"
            elif col in ['下降率(ft/min)', '高距离(ft/nm)', '累积时间(s)']:
                line += f"{value:>12.0f}"
            elif col in ['下降梯度(%)']:
                line += f"{value:>12.2f}"
            elif col in ['燃油流量(kg/h)']:
                line += f"{value:>12.1f}"
            else:
                line += f"{value:>14.1f}"
        print(line)
    
    print("-" * 165)

def print_deceleration_segments_table(decel_segments):
    """打印减速段分析表格"""
    if decel_segments:
        print("\n减速段分析:")
        print("-" * 120)
        print(f"{'类型':<12}{'高度层':<10}{'减速前CAS':<12}{'减速后CAS':<12}{'距离(nm)':<12}{'时间(s)':<10}{'燃油(kg)':<12}{'TAS(kt)':<10}")
        print("-" * 120)
        
        for segment in decel_segments:
            print(f"{segment['type']:<12}FL{segment['FL']:<8}{segment['减速前CAS']:<12.1f}"
                  f"{segment['减速后CAS']:<12.1f}{segment['减速距离(nm)']:<12.1f}"
                  f"{segment['减速时间(s)']:<10.0f}{segment['减速燃油(kg)']:<12.1f}{segment['TAS(kt)']:<10.1f}")
        
        print("-" * 120)

def print_final_profile_table(df):
    """打印包含减速段的最终下降剖面表格"""
    print("\n添加减速段后的详细下降数据:")
    print("-" * 180)
    
    # 选择要显示的关键列
    display_columns = [
        'FL', '速度模式', '马赫数', 'CAS(kt)', 'TAS(kt)', 
        '下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)',
        '燃油流量(kg/h)', '累积距离(nm)', '累积时间(s)', '累积时间(min)', '累积燃油(kg)'
    ]
    
    # 计算分钟列用于显示
    df_display = df.copy()
    df_display['累积时间(min)'] = df['累积时间(s)'] / 60
    
    # 打印表头
    header = ""
    for col in display_columns:
        if col in ['FL', '马赫数', 'CAS(kt)', 'TAS(kt)']:
            header += f"{col:>8}"
        elif col in ['速度模式']:
            header += f"{col:>12}"
        elif col in ['下降率(ft/min)', '下降梯度(%)', '高距离(ft/nm)', '燃油流量(kg/h)']:
            header += f"{col:>12}"
        elif col in ['累积时间(s)']:
            header += f"{col:>12}"
        elif col in ['累积时间(min)']:
            header += f"{col:>12}"
        else:
            header += f"{col:>14}"
    print(header)
    print("-" * 180)
    
    # 打印数据行
    for _, row in df_display.iterrows():
        line = ""
        for col in display_columns:
            value = row[col]
            if col in ['FL']:
                line += f"{value:>8.0f}"
            elif col in ['马赫数']:
                line += f"{value:>8.3f}"
            elif col in ['CAS(kt)', 'TAS(kt)']:
                line += f"{value:>8.1f}"
            elif col in ['速度模式']:
                line += f"{str(value):>12}"
            elif col in ['下降率(ft/min)', '高距离(ft/nm)', '累积时间(s)']:
                line += f"{value:>12.0f}"
            elif col in ['累积时间(min)']:
                line += f"{value:>12.1f}"
            elif col in ['下降梯度(%)']:
                line += f"{value:>12.2f}"
            elif col in ['燃油流量(kg/h)']:
                line += f"{value:>12.1f}"
            else:
                line += f"{value:>14.1f}"
        print(line)
    
    print("-" * 180)

def print_summary(summary):
    """打印下降剖面总结信息（同时显示秒和分钟）"""
    print(f"\n下降剖面总结:")
    print(f"总下降距离: {summary['Descent Distance (nm)']} nm")
    print(f"总下降时间: {summary['Descent Time (s)']} 秒 ({summary['Descent Time (s)']/60:.1f} 分钟)")
    print(f"总燃油消耗: {summary['Fuel Consumption (kg)']} kg")
    print(f"平均燃油流量: {summary['Average Fuel Flow (kg/h)']} kg/h")
    print(f"平均下降梯度: {summary['Average Descent Gradient (%)']} %")
    print(f"高距比: {summary['Altitude-to-Distance Ratio (ft/nm)']} ft/nm")
    print("=" * 80)

def print_final_summary(new_summary, basic_summary):
    """打印最终总结信息，包含与基础剖面的对比"""
    print(f"\n添加减速段后的下降剖面总结:")
    distance_increase = new_summary['Descent Distance (nm)'] - basic_summary['Descent Distance (nm)']
    time_increase = new_summary['Descent Time (s)'] - basic_summary['Descent Time (s)']
    fuel_increase = new_summary['Fuel Consumption (kg)'] - basic_summary['Fuel Consumption (kg)']
    
    print(f"总下降距离: {new_summary['Descent Distance (nm)']} nm (增加{distance_increase:.1f}nm)")
    print(f"总下降时间: {new_summary['Descent Time (s)']} 秒 ({new_summary['Descent Time (s)']/60:.1f} 分钟) (增加{time_increase:.0f}秒/{time_increase/60:.1f}分钟)")
    print(f"总燃油消耗: {new_summary['Fuel Consumption (kg)']} kg (增加{fuel_increase:.1f}kg)")
    print(f"平均下降梯度: {new_summary['Average Descent Gradient (%)']} %")
    print(f"高距比: {new_summary['Altitude-to-Distance Ratio (ft/nm)']} ft/nm")
    print("=" * 80)

def calculate_complete_descent_profile(cruise_fl, target_fl, aircraft_mass, 
                                     descent_mach, high_cas, ac_model, 
                                     low_cas=None, low_cas_fl=100, 
                                     constant_cas=None, include_deceleration=True,
                                     print_details=True):
    """
    计算完整的下降剖面（包括基础剖面和减速段）
    
    参数:
    cruise_fl: 巡航高度层 (FL)
    target_fl: 目标高度层 (FL)
    aircraft_mass: 飞机重量 (kg)
    descent_mach: 下降马赫数
    high_cas: 高高度CAS (kt)
    ac_model: 飞机型号
    low_cas: 低高度CAS (kt)
    low_cas_fl: 低高度CAS开始的高度层 (FL)
    constant_cas: 恒定CAS下降速度 (kt)
    include_deceleration: 是否包含减速段
    print_details: 是否打印详细信息
    
    返回:
    基础总结、完整总结、基础DataFrame、完整DataFrame、减速段列表
    """
    # 计算基础下降剖面
    basic_summary, basic_df, flight_levels = calculate_basic_descent_profile(
        cruise_fl, target_fl, aircraft_mass, descent_mach, high_cas, ac_model,
        low_cas, low_cas_fl, constant_cas, print_details
    )
    
    if include_deceleration:
        # 添加减速段
        final_summary, final_df, decel_segments = apply_deceleration_segments(
            basic_summary, basic_df, flight_levels, high_cas, low_cas, low_cas_fl, print_details
        )
        return basic_summary, final_summary, basic_df, final_df, decel_segments
    else:
        return basic_summary, None, basic_df, None, []


In [7]:

# 使用示例
if __name__ == "__main__":
    # 示例计算
    basic_summary, final_summary, basic_df, final_df, decel_segments = calculate_complete_descent_profile(
        cruise_fl=370,
        target_fl=30,
        aircraft_mass=60000,
        descent_mach=0.73,
        high_cas=250,
        ac_model="A320-232",
        low_cas=220,
        low_cas_fl=150,
        include_deceleration=True,
        print_details=True
    )


基础下降剖面分析: A320-232
飞机重量: 60.0 吨
下降剖面: 0.73M/250kt/220kt@FL150
巡航高度: FL370 (37,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL342 (34,200 ft)

基础下降数据 - 0.73M/250kt/220kt@FL150:
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)       累积时间(s)      累积燃油(kg)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
     370   Mach 0.73   0.730   234.7   418.7       -2234        5.27         320       535.0           0.0           0           0.0
     360   Mach 0.73   0.730   240.3   418.9       -2425        5.72         347       533.3           3.0          26           3.8
     350   Mach 0.73   0.730   245.9   420.8       -2475        5.81         353       531.4       

In [8]:
basic_summary, final_summary, basic_df, final_df, decel_segments = calculate_complete_descent_profile(
        cruise_fl=370,
        target_fl=30,
        aircraft_mass=60000,
        descent_mach=0.80,
        high_cas=310,
        ac_model="A320-232",
        low_cas=250,
        low_cas_fl=100,
        include_deceleration=True,
        print_details=True
    )


基础下降剖面分析: A320-232
飞机重量: 60.0 吨
下降剖面: 0.8M/310kt/250kt@FL100
巡航高度: FL370 (37,000 ft)
目标高度: FL30 (3,000 ft)
过渡高度: FL290 (29,000 ft)

基础下降数据 - 0.8M/310kt/250kt@FL100:
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
      FL        速度模式     马赫数 CAS(kt) TAS(kt) 下降率(ft/min)     下降梯度(%)  高距离(ft/nm)  燃油流量(kg/h)      累积距离(nm)       累积时间(s)      累积燃油(kg)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
     370    Mach 0.8   0.800   259.7   458.9       -2748        5.91         359       560.3           0.0           0           0.0
     360    Mach 0.8   0.800   265.8   459.0       -3072        6.61         401       556.6           2.6          21           3.2
     350    Mach 0.8   0.800   271.9   461.1       -3176        6.80         413       552.4         